In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, month, dayofmonth

In [11]:
spark = SparkSession \
    .builder \
    .appName("PySpark") \
    .getOrCreate()

In [38]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType
from pyspark.sql import types

df = spark.read.format('csv').option("delimiter", "\t").option("header", "true").load("data/election_data.csv").toDF("index", "region", "tik_name", "uik_number",
             "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", 
             "11", "12", "13", "14", "15", "16", "17", "18", "19", "20")
df.show(10)

+-----+--------------------+------------+----------+----+----+---+----+---+---+---+----+---+----+---+---+---+---+---+----+---+---+---+---+
|index|              region|    tik_name|uik_number|   1|   2|  3|   4|  5|  6|  7|   8|  9|  10| 11| 12| 13| 14| 15|  16| 17| 18| 19| 20|
+-----+--------------------+------------+----------+----+----+---+----+---+---+---+----+---+----+---+---+---+---+---+----+---+---+---+---+
|    0| Республика Адыге...|1 Адыгейская|    УИК №1|2256|2181|  0|2107| 62| 12| 62|2107|  3|2166|  0|  0|  0|137| 32|1977| 14|  0|  1|  5|
|    1| Республика Адыге...|1 Адыгейская|    УИК №2|2700|2633|  0|2575| 41| 17| 41|2575| 22|2594|  0|  0| 15| 86| 65|2389| 13|  5|  6| 15|
|    2| Республика Адыге...|1 Адыгейская|    УИК №3|2858|2752|  0|2664| 75| 13| 75|2664|  5|2734|  0|  0|  1| 62| 13|2645|  6|  3|  4|  0|
|    3| Республика Адыге...|1 Адыгейская|    УИК №4|2066|2034|  0|1857|142| 35|142|1857| 21|1978|  0|  0|  5|288| 12|1642| 21|  6|  2|  2|
|    4| Республика Адыге...

In [55]:
import pyspark.sql.functions as F

regions_appearance = df.groupBy('region').agg((F.sum('4') / F.sum('2')).alias('appearance')).sort(F.desc('appearance'))
regions_appearance.show()

+--------------------+------------------+
|              region|        appearance|
+--------------------+------------------+
| Чеченская Респуб...|0.9237943323561534|
| Кабардино-Балкар...|0.8751962690631299|
| Республика Дагестан|0.8618083655713163|
| Республика Север...|0.8573365153263826|
| Республика Ингуш...|0.8456886972860325|
| Карачаево-Черкес...|0.8374121327086533|
|   город Севастополь|0.8112282770821049|
| Ямало-Ненецкий а...|0.7943875261640483|
| Кемеровская область|0.7893942992507781|
|     Республика Тыва|0.7877189303133453|
|   Тюменская область| 0.783086092265614|
|     Республика Крым|0.7823055579528453|
| Республика Татар...|0.7691193222663378|
| Республика Башко...| 0.762702842604394|
| Республика Мордовия|0.7500784190715182|
| Чукотский автоно...|0.7490556690816769|
|    Брянская область|0.7283976086224317|
|  Республика Бурятия| 0.724575722243814|
|  Краснодарский край| 0.716545580206834|
| Республика Адыге...|0.7151332964164228|
+--------------------+------------

In [82]:
sobchak = df.filter(F.col('17') > 300).groupBy('region', 'uik_number').agg(F.max('17'))
sobchak.show()

+------+----------+-------+
|region|uik_number|max(17)|
+------+----------+-------+
+------+----------+-------+



In [114]:
differences = df.groupBy('region').agg(F.abs(F.max('2') - F.min('2')).alias('diff'))\
#     .agg(F.max('diff'))
differences.show()

+--------------------+-----+
|              region| diff|
+--------------------+-----+
|    Липецкая область|870.0|
| Республика Ингуш...| 31.0|
| Ставропольский край|870.0|
|   Иркутская область|890.0|
|   Орловская область|890.0|
| Республика Саха ...|898.0|
|    Брянская область|890.0|
| Карачаево-Черкес...| 17.0|
| Чукотский автоно...|756.0|
|    Республика Алтай|895.0|
|    Амурская область|895.0|
| Новосибирская об...|896.0|
|     Республика Тыва| 43.0|
|   Красноярский край|896.0|
|     Камчатский край|897.0|
|  Республика Бурятия|891.0|
| Чувашская Респуб...|897.0|
| Еврейская автоно...| 30.0|
| Нижегородская об...|897.0|
| Удмуртская Респу...|892.0|
+--------------------+-----+
only showing top 20 rows



In [118]:
d = df.groupBy('region').agg(F.variance('4'))
d.show()

+--------------------+------------------+
|              region|       var_samp(4)|
+--------------------+------------------+
|    Липецкая область|  235136.687612936|
| Республика Ингуш...|349489.99474023184|
| Ставропольский край| 291515.8075166363|
|   Иркутская область|189089.51083332888|
|   Орловская область|219259.84429035857|
| Республика Саха ...|208031.99857589803|
|    Брянская область|294871.38655167475|
| Карачаево-Черкес...|271935.51241434267|
| Чукотский автоно...|193242.96498316491|
|    Республика Алтай| 139849.6498919791|
|    Амурская область|173755.91486058763|
| Новосибирская об...|241561.71941852986|
|     Республика Тыва|366036.62373145943|
|   Красноярский край| 236772.0892371602|
|  Республика Бурятия|194877.62826567012|
|     Камчатский край| 288771.1985185485|
| Чувашская Респуб...|223338.07897832015|
| Еврейская автоно...|135965.78272748852|
| Нижегородская об...|242162.36506902066|
| Удмуртская Респу...|190925.99049096514|
+--------------------+------------